# 一、配置model

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = "glm-4-flash",
    openai_api_base = "https://open.bigmodel.cn/api/paas/v4/",
    api_key = "8d79e8ba89ad4a9aa0b45686521c7616.zCFcoSOnlV0Kb8LU"
)

# 二、编写Tools

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

In [7]:
from langchain.tools import tool
from pandasql import sqldf

import pandas as pd


@tool
def simulate_db_operation(sql):
    """
    根据sql语句操作数据库中的数据
    """
    my_table = pd.DataFrame({
        "name": ["AD", "LBJ", "AR", "PG", "JH", "KL"],
        "age": ["30", "41", "26", "34", "36", "33"]
    })

    result = sqldf(sql)

    return result


In [ ]:
# 简单测试一下
from pprint import pprint

tools = [DuckDuckGoSearchRun(), simulate_db_operation]
model_with_tools = model.bind_tools(tools)

# 无需工具版
response = model_with_tools.invoke("印度的首都是哪里？")
pprint(dict(response))

{'additional_kwargs': {'refusal': None},
 'content': '印度的首都是新德里（New Delhi）。它是印度联邦的首都，同时也是德里联邦属地的首府。',
 'id': 'lc_run--019af276-f48d-78b3-87fb-a417b7814bee-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'id': '20251206150114ff88f87dee504ecc',
                       'logprobs': None,
                       'model_name': 'glm-4-flash',
                       'model_provider': 'openai',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 25,
                                       'completion_tokens_details': None,
                                       'prompt_tokens': 268,
                                       'prompt_tokens_details': None,
                                       'total_tokens': 293}},
 'tool_calls': [],
 'type': 'ai',
 'usage_metadata': {'input_token_details': {},
                    'input_tokens': 268,
                    'output_token_detail

In [11]:
# 需要工具版
response = model_with_tools.invoke("今天广州的天气如何？")
pprint(dict(response))

{'additional_kwargs': {'refusal': None},
 'content': '这个问题需要使用天气工具来回答。{"index":0,"finish_reason":"tool_calls","delta":{"role":"assistant","content":null,"reasoning_content":null,"audio":null,"tool_calls":[{"id":"call_20251206150211be622f4bb65c43ca_0","index":0,"type":"function","function":{"name":"duckduckgo_search","arguments":"{\\"query\\": '
            '\\"广州今天天气\\"}","outputs":null},"code_interpreter":null,"retrieval":null,"mcp":null,"drawing_tool":null,"web_browser":null,"search_intent":null,"search_result":null}],"tool_call_id":null,"attachments":null,"metadata":null}}',
 'id': 'lc_run--019af277-d6bb-76e1-a541-66e437b3935a-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': '20251206150211be622f4bb65c43ca',
                       'logprobs': None,
                       'model_name': 'glm-4-flash',
                       'model_provider': 'openai',
                       'system_fingerprint': None,
    

In [12]:
# 看看操纵一下数据库
response = model_with_tools.invoke("帮我王数据库中插入一条内容，name是TM，age是24")
pprint(dict(response))

{'additional_kwargs': {'refusal': None},
 'content': '',
 'id': 'lc_run--019af27a-af48-7f11-91f9-5b6675837b00-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': '2025120615051824ee943c56134821',
                       'logprobs': None,
                       'model_name': 'glm-4-flash',
                       'model_provider': 'openai',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 25,
                                       'completion_tokens_details': None,
                                       'prompt_tokens': 278,
                                       'prompt_tokens_details': None,
                                       'total_tokens': 303}},
 'tool_calls': [{'args': {'sql': "INSERT INTO People (name, age) VALUES ('TM', "
                                 '24)'},
                 'id': 'call_-8093604595771753754',
                 'name': 's

# 三、手动实现Agent

In [19]:
def agent_handmade(query: str, model: ChatOpenAI, tools: list[tool]):
    model_with_tools = model.bind_tools(tools)
    model_output = model_with_tools.invoke(query)
    tool_response = call_tool(model_output, tools)
    final_response = model.invoke(
        f"origin query: {query} \n\n\n tool response: {tool_response}"
    )
    return final_response

def call_tool(model_output, tools):
    tools_map = {tool.name.lower(): tool for tool in tools}
    tools_response = {}
    for tool in model_output.tool_calls:
        tool_name = tool["name"]
        tool_args = tool["args"]
        tool_instance = tools_map[tool_name]
        tool_response = tool_instance.invoke(*tool_args.values())
        tools_response[tool_name] = tool_response
    return tools_response

In [21]:
agent_handmade("数据库my_table中年龄低于30的有几人？", model, tools).content

'根据工具的响应，我们可以推断出数据库`my_table`中年龄低于30的人有1人。工具的响应使用了`COUNT(*)`函数来统计记录数，其中第二行数据表示统计结果为1。这表明满足条件“年龄低于30”的记录有1条。'

# 四、LangGraph实现Agent

In [31]:
from langchain.agents import create_agent

agent = create_agent(model, tools)

In [32]:
from langchain_core.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage("数据库my_table中年龄高于30的有几人？")]}
)
response

{'messages': [HumanMessage(content='数据库my_table中年龄高于30的有几人？', additional_kwargs={}, response_metadata={}, id='278a4ecd-4768-4a61-8108-018a5108b9e9'),
  AIMessage(content='为了回答这个问题，我需要查询数据库my_table中年龄高于30的记录数量。以下是查询语句：```\nSELECT COUNT(*) FROM my_table WHERE age > 30;\n```接下来我将使用工具来执行这个查询。{"index":0,"finish_reason":"tool_calls","delta":{"role":"assistant","content":null,"reasoning_content":null,"audio":null,"tool_calls":[{"id":"call_2025120615523204a1eb0e382944eb_0","index":0,"type":"function","function":{"name":"simulate_db_operation","arguments":"{\\"sql\\": \\"SELECT COUNT(*) FROM my_table WHERE age > 30;\\"}","outputs":null},"code_interpreter":null,"retrieval":null,"mcp":null,"drawing_tool":null,"web_browser":null,"search_intent":null,"search_result":null}],"tool_call_id":null,"attachments":null,"metadata":null}}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 274, 'total_tokens': 344, 'completion_tokens_details': N

In [33]:
response = agent.invoke(
    {"messages": [HumanMessage("今天广州的天气怎么样")]}
)
response

{'messages': [HumanMessage(content='今天广州的天气怎么样', additional_kwargs={}, response_metadata={}, id='c52a5fde-d5c1-4314-ae72-c63b4eccf25e'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 268, 'total_tokens': 282, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'id': '2025120615530389f46cd0c05340d6', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019af2a6-68e9-79b1-8ace-e169a3dcecc1-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': '广州今天天气'}, 'id': 'call_-8093603839857513826', 'type': 'tool_call'}], usage_metadata={'input_tokens': 268, 'output_tokens': 14, 'total_tokens': 282, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content='广 州 天 气 预报15天. 广 州 天 气 预报30天. 广 州 天 气 预报 广 州 一周 天 气 广 州 明 天 天 气 广 州 历史 天 气 . 今 天 北京仍有降雨最高 气 温仅14℃ 明后天雨水停歇 气 温回升. 202